# get differential genes from deseq from data from Eileen White's lab

In [1]:
library( "DESeq2" )
library('tximport')
library('stringr')
library(dplyr)

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min



Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomicRanges

Loading required package: GenomeInfoDb

Loading required package: SummarizedExperiment

Loading required package: MatrixGe

In [51]:
out_file_path <- './'
data_dir <- paste(out_file_path,'data/white_lab/', sep = '')
counts_dir <- paste(data_dir,'counts_filtered.csv', sep = '')
out_dir <- paste(out_file_path, 'white_results/', sep = '')
metadata_liver <- paste(data_dir,'metadata_liver.csv', sep = '')

**read in scRNA data**

In [68]:
counts <- read.csv(counts_dir, sep = ',', row.names = 1) # scrna
counts_colnames <- read.csv(paste(data_dir,'counts_colnames.csv', sep = ''))
colnames(counts) <- counts_colnames$X0
head(counts)

,12DarkATG7KOfemale1911-21-GNP,12DarkATG7KOfemale1911-22-iWAT,12DarkATG7KOfemale1911-23-BAT,12DarkATG7KOfemale1911-24-Liver,12DarkATG7KOfemale1911-25-Cerebellum,12DarkATG7KOfemale1914-26-GNP,12DarkATG7KOfemale1914-27-Liver,12DarkATG7KOfemale1914-28-BAT,12DarkATG7KOfemale1914-29-Cerebellum,12DarkATG7KOfemale1914-30-iWAT,⋯,6LightATG7WT1977-143-BAT,6LightATG7WT1977-144-Liver,6LightATG7WT2040-125-Cerebellum,6LightATG7WT2040-126-BAT,6LightATG7WT2040-127-GNP,6LightATG7WT2040-128-Liver,6LightATG7WT2041-77-Liver,6LightATG7WT2041-78-BAT,6LightATG7WT2041-79-Cerebellum,6LightATG7WT2041-80-GNP
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
0610005C13Rik,15,12,12,2165,21,22,2125,13,19,23,⋯,5,2534,21,22,12,4015,3223,21,18,15
0610009B22Rik,306,419,430,353,198,373,388,425,201,360,⋯,373,318,272,445,581,460,402,199,264,583
0610009E02Rik,3,20,11,2,14,4,5,4,17,18,⋯,11,9,11,12,2,12,2,9,18,3
0610009L18Rik,246,103,31,14,24,309,13,25,11,75,⋯,14,2,46,46,262,26,18,45,50,246
0610010F05Rik,277,361,337,265,552,370,274,325,449,302,⋯,528,350,875,394,417,430,217,243,743,360
0610010K14Rik,11,45,9,12,64,11,15,6,50,46,⋯,21,17,52,31,13,27,20,35,52,15


In [69]:
metadata <- read.csv(metadata_liver)
metadata_colnames <- read.csv(paste(data_dir,'metadata_colnames_liver.csv', sep = ''))
rownames(metadata) <- metadata_colnames$X0
metadata

,X,organ,samp,time,genotype,gender
,<chr>,<chr>,<int>,<chr>,<chr>,<chr>
12DarkATG7KOfemale1911-24-Liver,12DarkATG7KOfemale1911-24-Liver,Liver,24,Dark,KO,female
12DarkATG7KOfemale1914-27-Liver,12DarkATG7KOfemale1914-27-Liver,Liver,27,Dark,KO,female
12DarkATG7KOmale1935-36-Liver,12DarkATG7KOmale1935-36-Liver,Liver,36,Dark,KO,male
12DarkATG7WTmale1981-42-Liver,12DarkATG7WTmale1981-42-Liver,Liver,42,Dark,WT,male
12DarkATG7WTmale1983-32-Liver,12DarkATG7WTmale1983-32-Liver,Liver,32,Dark,WT,male
12LightATG7KOfemale1390-20-Liver,12LightATG7KOfemale1390-20-Liver,Liver,20,Light,KO,female
12LightATG7KOfemale1392-8-Liver,12LightATG7KOfemale1392-8-Liver,Liver,8,Light,KO,female
12LightATG7WTfemale2024-2-Liver,12LightATG7WTfemale2024-2-Liver,Liver,2,Light,WT,female
2DarkATG7KO1904-51-Liver,2DarkATG7KO1904-51-Liver,Liver,51,Dark,KO,male


In [70]:
counts_liver = counts[,metadata_colnames$X0]

In [71]:
head(counts_liver)

,12DarkATG7KOfemale1911-24-Liver,12DarkATG7KOfemale1914-27-Liver,12DarkATG7KOmale1935-36-Liver,12DarkATG7WTmale1981-42-Liver,12DarkATG7WTmale1983-32-Liver,12LightATG7KOfemale1390-20-Liver,12LightATG7KOfemale1392-8-Liver,12LightATG7WTfemale2024-2-Liver,2DarkATG7KO1904-51-Liver,2DarkATG7KO2581-56-Liver,⋯,6DarkATG7WT2015-93-Liver,6DarkATG7WT2033-102-Liver,6DarkATG7WT2046-100-Liver,6LightATG7KO1921-122-Liver,6LightATG7KO1922-136-Liver,6LightATG7KO1956-133-Liver,6LightATG7WT1975-86-Liver,6LightATG7WT1977-144-Liver,6LightATG7WT2040-128-Liver,6LightATG7WT2041-77-Liver
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
0610005C13Rik,2165,2125,2922,2347,2128,2194,2322,6111,2113,2635,⋯,2140,2546,2429,2230,1822,2189,2669,2534,4015,3223
0610009B22Rik,353,388,410,325,298,361,478,790,301,304,⋯,308,324,298,535,266,213,409,318,460,402
0610009E02Rik,2,5,10,6,2,1,9,11,8,6,⋯,4,7,5,12,5,4,14,9,12,2
0610009L18Rik,14,13,7,4,3,18,16,45,5,7,⋯,6,8,5,23,1,6,7,2,26,18
0610010F05Rik,265,274,168,213,257,215,298,445,154,124,⋯,230,350,300,428,284,62,320,350,430,217
0610010K14Rik,12,15,36,18,13,12,27,117,29,33,⋯,27,18,20,15,19,24,18,17,27,20


In [72]:
head(metadata)

,X,organ,samp,time,genotype,gender
,<chr>,<chr>,<int>,<chr>,<chr>,<chr>
12DarkATG7KOfemale1911-24-Liver,12DarkATG7KOfemale1911-24-Liver,Liver,24,Dark,KO,female
12DarkATG7KOfemale1914-27-Liver,12DarkATG7KOfemale1914-27-Liver,Liver,27,Dark,KO,female
12DarkATG7KOmale1935-36-Liver,12DarkATG7KOmale1935-36-Liver,Liver,36,Dark,KO,male
12DarkATG7WTmale1981-42-Liver,12DarkATG7WTmale1981-42-Liver,Liver,42,Dark,WT,male
12DarkATG7WTmale1983-32-Liver,12DarkATG7WTmale1983-32-Liver,Liver,32,Dark,WT,male
12LightATG7KOfemale1390-20-Liver,12LightATG7KOfemale1390-20-Liver,Liver,20,Light,KO,female


# run deseq

In [73]:
dds <- DESeqDataSetFromMatrix(countData=counts_liver, 
                              colData=metadata, 
                              design=~gender + time + genotype)

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”


In [74]:
startTime <- Sys.time()
dds <- DESeq(dds)
print(Sys.time() - startTime)

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 103 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



Time difference of 23.74883 secs


In [77]:
res <- results(dds)

In [78]:
resultsNames(dds)

[1] "Intercept"             "gender_male_vs_female" "time_Light_vs_Dark"   
[4] "genotype_WT_vs_KO"

In [79]:
write.csv(res, file=paste(out_dir, 'deseq_genes_liver_WT_vs_KO.csv', sep = ''))

In [80]:
write.csv(results(dds, name = 'gender_male_vs_female'), file=paste(out_dir, 'deseq_genes_liver_male_vs_female.csv', sep = ''))

In [81]:
write.csv(results(dds, name = 'time_Light_vs_Dark'), file=paste(out_dir, 'deseq_genes_liver_light_vs_dark.csv', sep = ''))